## Deliverable 3. Create a Travel Itinerary Map.

In [175]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [176]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("..\Weather_Database\weatherpy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name,Current Description
0,0,Makokou,GA,295.07,0.5738,12.8642,WAMY RESIDENCE HOTEL,NaN
1,2,Foumban,CM,297.62,5.7291,10.9001,Hotel Pekassa De Karche,NaN
2,3,Nola,IT,285.93,40.9220,14.5329,Pamaran Hotel,NaN
3,6,Paoua,CF,299.27,7.2427,16.4406,Quartier Paoua Elien,NaN
4,14,Mao,ES,285.15,39.8885,4.2658,ARTIEM Capri,NaN


In [177]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
clean_hotel_df = vacation_df
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,0.5738,12.8642
1,5.7291,10.9001
2,40.9220,14.5329
3,7.2427,16.4406
4,39.8885,4.2658
...,...,...
114,69.3000,139.9000
115,68.7500,161.3000
116,64.7500,177.4833
117,53.9606,105.8817


In [178]:
# 4a. Add a marker layer for each city to the map.
import gmaps
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = clean_hotel_df["Max Temp"]

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [179]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
vacation_df = clean_hotel_df.loc[clean_hotel_df['Country'] == 'RU']
# vacation_df.head(4)
vacation_start = vacation_df.loc[vacation_df['City'] == 'Divnoye']
# vacation_start
vacation_end = vacation_df.loc[vacation_df['City'] == 'Divnoye']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Derbent']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Onguday'] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Kuytun'] 

In [180]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
import numpy as np

start = pd.DataFrame(vacation_start, columns=['Lat', 'Lng'])
stop1 = pd.DataFrame(vacation_stop1, columns=['Lat', 'Lng'])
stop2 = pd.DataFrame(vacation_stop2, columns=['Lat', 'Lng'])
stop3 = pd.DataFrame(vacation_stop3, columns=['Lat', 'Lng'])
itinerary_concat = pd.concat([start, stop1, stop2, stop3] ,ignore_index=True)
itinerary_concat = pd.DataFrame(itinerary_concat).to_numpy
a_list = list()
a_list.append(start.values.tolist())
a_list.append(stop1.values.tolist())
a_list.append(stop2.values.tolist())
a_list.append(stop3.values.tolist())

# print(itinerary_concat)
# print(a_list[0][0][0])
# print(another_list)

In [181]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
Divnoye = (a_list[0][0][0], a_list[0][0][1]) #(45.9089, 43.3547)
Divnoye= (a_list[0][0][0], a_list[0][0][0]) #(45.9089,    43.3547)
Derbent = (a_list[1][0][0], a_list[1][0][1]) #(42.0678, 48.2899	)
Kuytun = (a_list[2][0][0], a_list[2][0][1]) #(54.3419, 101.5036)
Onguday =  (a_list[3][0][0], a_list[3][0][1]) #(50.7500,	86.1500)
print(Divnoye,Derbent,Kuytun,Onguday)
fig = gmaps.figure()
Itinerary = gmaps.directions_layer(Divnoye, Divnoye, waypoints=[Kuytun, Onguday], travel_mode="DRIVING")
fig.add_layer(Itinerary)
fig


(45.9089, 45.9089) (42.0678, 48.2899) (50.75, 86.15) (54.3419, 101.5036)


Figure(layout=FigureLayout(height='420px'))

In [182]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name,Current Description
0,38,Divnoye,RU,276.56,45.9089,43.3547,Motel Manych-Gudilo,NaN
1,89,Derbent,RU,281.55,42.0678,48.2899,Elit,NaN
2,103,Onguday,RU,273.31,50.7500,86.1500,"Gostinitsa ""Motel'""",NaN
3,98,Kuytun,RU,263.80,54.3419,101.5036,Gostinitsa Tsentral'naya,NaN


In [183]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Motel Manych-Gudilo</dd>\n<dt>City</dt><dd>Divnoye</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Description</dt><dd>nan</dd>\n<dt>Max Temp</dt><dd>276.56 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Elit</dd>\n<dt>City</dt><dd>Derbent</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Description</dt><dd>nan</dd>\n<dt>Max Temp</dt><dd>281.55 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Gostinitsa "Motel\'"</dd>\n<dt>City</dt><dd>Onguday</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Description</dt><dd>nan</dd>\n<dt>Max Temp</dt><dd>273.31 °F</dd>\n</dl>\n',
 "\n<dl>\n<dt>Hotel Name</dt><dd>Gostinitsa Tsentral'naya</dd>\n<dt>City</dt><dd>Kuytun</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Description</dt><dd>nan</dd>\n<dt>Max Temp</dt><dd>263.8 °F</dd>\n</dl>\n"]

In [184]:
# 11a. Add a marker layer for each city to the map.
import gmaps
locations = itinerary_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = itinerary_df["Max Temp"]

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))